# Section 1-1 - Basic Neural Network - Titanic

In the previous section, we simply iterated through randomly-generated matrices and chose the best-performing one. We build on this approach by reducing loss in a systematic way via stochastic gradient descent. In particular, we'll be using TensorFlow, an open source library developed by Google, and Keras, a high-level wrapper on top of TensorFlow.

In [23]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from time import time

np.random.seed(1337)

df = pd.read_csv('../data/titanic.csv')
df_test1 = pd.read_csv('../data/test.csv')

In [58]:
df_train = df.iloc[:712, :]

scaler = StandardScaler()
# p= pd.get_dummies(df_train1[['Sex']])
# df_train = pd.concat([p,df_train1[['Class','Age','Fare']]], axis=1)
# df_train

features = ['Class', 'Sex', 'Age', 'Fare']

X_train = scaler.fit_transform(df_train[features].values)
y_train = df_train['Survived'].values
y_train_onehot = pd.get_dummies(df_train['Survived']).values

In [59]:
df_test = df.iloc[712:, :]

X_test = scaler.transform(df_test[features].values)
y_test = df_test['Survived'].values


## Benchmark

In [66]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(random_state=0, verbose=3)
model = model.fit(X_train, df_train['Survived'].values)

y_prediction = model.predict(X_test)


#df_test1[[]]
#X_validation
k = df_test1.replace({'male': 1}, regex=False)
l = k.replace({'female': 0}, regex=False)
l
#df_test1

# p= pd.get_dummies(df_test1[['Sex']])
# X_vali = pd.concat([p,df_test1[['Pclass','Age','Fare']]], axis=1)
X_vali = l[['Pclass', 'Sex','Age','Fare']]
X_vali.shape
# Make predictions on validation dataset
X_vali

# print (predictions)
# predictions.shape
print (X_vali.isnull().any())
X_vali.isnull().sum()

# X_vali
# X_valid = X_vali.fillna(0)
# X_valid.isnull().any()

# y_prediction_test = model.predict(X_valid)

# y_prediction_test

# print ("\naccuracy", np.sum(y_prediction == y_test) / float(len(y_test)))

# # submission = pd.DataFrame({
# #         "PassengerId": df_test1["PassengerId"],
# #         "Survived": y_prediction_test
# #     })
# # submission.to_csv('titanic_random.csv', index=False)


building tree 1 of 10
building tree 2 of 10
building tree 3 of 10
building tree 4 of 10
building tree 5 of 10
building tree 6 of 10
building tree 7 of 10
building tree 8 of 10
building tree 9 of 10
building tree 10 of 10
Pclass    False
Sex       False
Age        True
Fare       True
dtype: bool


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished


Pclass     0
Sex        0
Age       86
Fare       1
dtype: int64

## 1-layer Neural Network

Instead of generating a linear stack of layers with Numpy, we'll be implementing our model using Keras. We initialize our model, add a layer that inputs vectors of length 4 and outputs vectors of length 2, and finally add a softmax layer. We configure the learning process in the compilation step by specifying the optimizer, loss function and performance metric.

Stochastic gradient descent acts by changing the weights gradually in the 'direction' that decreases the average loss. In other words, a particular weight would be increased if acts to decrease loss, or the weight decreased if it acts to increase loss. TensorFlow does the heavy-lifting by efficiently handling these numerical computations under the hood. A simple example of stochastic gradient descent is illustrated in the Appendix.

In [9]:
from keras.models import Sequential
from keras.layers import Dense, Activation

start = time()

model = Sequential()
model.add(Dense(input_dim=4, output_dim=2))
model.add(Activation("softmax"))

model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

model.fit(X_train, y_train_onehot)

print ('\ntime taken %s seconds' % str(time() - start))

Using TensorFlow backend.


Epoch 1/10
712/712 [==============================] - 0s - loss: 0.6138 - acc: 0.7261     
Epoch 2/10
712/712 [==============================] - 0s - loss: 0.5947 - acc: 0.7360     
Epoch 3/10
712/712 [==============================] - 0s - loss: 0.5791 - acc: 0.7402     

In [11]:
y_prediction = model.predict_classes(X_test)
print ("\n\naccuracy", np.sum(y_prediction == y_test) / float(len(y_test)))

 32/179 [====>.........................] - ETA: 0s

accuracy 0.782122905028


We notice that the loss reduces systematically as the model 'learns' from the data. The rate of loss reduction, however, seems to indicate that loss could be further reduced.

## 2-layer Neural Network

In [12]:
start = time()

model = Sequential()
model.add(Dense(input_dim=4, output_dim=100))
model.add(Dense(output_dim=2))
model.add(Activation("softmax"))

model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

model.fit(X_train, y_train_onehot)

print ('\ntime taken %s seconds' % str(time() - start))

Epoch 1/10
712/712 [==============================] - 0s - loss: 0.5477 - acc: 0.7612     
Epoch 2/10
712/712 [==============================] - 0s - loss: 0.5183 - acc: 0.7711     
Epoch 3/10
712/712 [==============================] - 0s - loss: 0.5014 - acc: 0.7753     

In [13]:
y_prediction = model.predict_classes(X_test)
print ("\n\naccuracy", np.sum(y_prediction == y_test) / float(len(y_test)))

 32/179 [====>.........................] - ETA: 0s

accuracy 0.810055865922


The loss reduction 'flattens out' more compared to the 1-layer example, and the accuracy improves to 81%.

## 3-layer Neural Network

In [14]:
start = time()

model = Sequential()
model.add(Dense(input_dim=4, output_dim=100))
model.add(Dense(output_dim=100))
model.add(Dense(output_dim=2))
model.add(Activation("softmax"))

model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

model.fit(X_train, y_train_onehot)

print ('\ntime taken %s seconds' % str(time() - start))

Epoch 1/10
712/712 [==============================] - 0s - loss: 0.5707 - acc: 0.7346     
Epoch 2/10
712/712 [==============================] - 0s - loss: 0.4980 - acc: 0.7809     
Epoch 3/10
712/712 [==============================] - 0s - loss: 0.4772 - acc: 0.7795     

In [15]:
y_prediction = model.predict_classes(X_test)
print ("\n\naccuracy", np.sum(y_prediction == y_test) / float(len(y_test)))

 32/179 [====>.........................] - ETA: 0s

accuracy 0.815642458101


While we're able to reduce loss on the training set a little further, the best performance obtained is merely comparable to our benchmark. Since the dataset is small, there isn't as much for the model to 'learn' from (or for that matter, predict on). 